### Collate brain ages from DenseNet, PHOTON, UCSD Multimodal and BrainAgeR

In [1]:
import os
import pandas as pd
import pyreadstat

path = os.path.join("..", "data_original", "Master_BAI_080720_with_051517completers_060117imagingmaster_genotypes_brainages_ENIGMA_FS_MERGEFK_30122020.sav")
df_photon_brainager, meta = pyreadstat.read_sav(path)

age_cols = [col for col in df_photon_brainager.columns if 'age' in col.lower()] # sanity check to see what columns contain age 
print(age_cols)

df_photon_brainager = df_photon_brainager[~df_photon_brainager[['ageprediction_BrainAgeR', 'age_prediction']].isna().any(axis=1)]
df_photon_brainager.shape
#df = df[(df["visit"] == "Baseline")].drop_duplicates()
 

['agevisit', 'ageenroll', 'Brain_Age', 'AgeDisc', 'Brain_age_quad', 'AgeDisc_Quad', 'Age2', 'PLS_BrainAge', 'PCA_BrainAge', 'Lasso_BrainAge', 'AgeBins', 'Age3', 'age_prediction', 'AgeofOnset', 'ageprediction_BrainAgeR', 'ageprediction_BrainAgeR_lowerCI', 'ageprediction_BrainAgeR_upperCI', 'greymatterVol_BrainAgeR', 'whitematterVol_BrainAgeR', 'CSFVol_BrainAgeR']


(117, 2335)

In [2]:
path = os.path.join("..", "data_processed", "bpad_ridge.csv")
df_multimodal = pd.read_csv(path)

df_multimodal =  df_multimodal.rename(columns={"BD#": "stdyptid"})
df_multimodal = df_multimodal[['stdyptid', 'ridge_BrainAge']]
full_df = df_photon_brainager.merge(df_multimodal, how='inner', on='stdyptid')
full_df.shape 

(117, 2336)

In [3]:
# subset_control_train = pd.read_csv('../data_processed/hc_train_age_proportion_match_bd.csv')
# subset_control_test = pd.read_csv('../data_processed/hc_test_age_proportion_match_bd.csv')
 

CNN DF

In [4]:
cnn_brain_age_path = os.path.join("..", "data_original", "master_bai_merge_cnn_brainage.csv")
df_cnn = pd.read_csv(cnn_brain_age_path, index_col= False)
df_cnn = df_cnn.drop_duplicates(subset="stdyptid")
df_cnn = df_cnn[['stdyptid', 'cnn_BrainAge']]

 
full_df = full_df.merge(df_cnn, how='inner', on='stdyptid')

C:\Users\hxng\AppData\Local\Temp\ipykernel_244\938450680.py:2: DtypeWarning: Columns (21,26,399,415,419,465,466,467,469,471,473,830,832,858,859,866,868,926,929) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cnn = pd.read_csv(cnn_brain_age_path, index_col= False)


In [5]:
full_df

,MatchedFK,Matched,subnum,dxgroupFK,dxgroup,anrtepiq_FK,stdyptid,visit,tag,status,...,greymatterVol_BrainAgeR,whitematterVol_BrainAgeR,CSFVol_BrainAgeR,filter_$,PLS_DiscFK,BrainPADFK,PLS_DiscFKxgroup,BrainPADFKxgroup,ridge_BrainAge,cnn_BrainAge
0,1.0,1.0,455346.0,0.0,0.0,104.64,BD178,0.0,184.0,199.0,...,0.633771,0.419431,0.214442,1.0,0.427237,-6.667611,0.000000,0.000000,NaN,45.92
1,1.0,1.0,523869.0,0.0,0.0,108.36,BD030,0.0,184.0,199.0,...,0.629238,0.383269,0.261224,1.0,3.007307,-7.340975,0.000000,0.000000,51.970783,48.94
2,0.0,0.0,10000514.0,0.0,0.0,119.40,BD222,0.0,184.0,199.0,...,0.641076,0.417520,0.575853,0.0,-0.493102,3.803285,0.000000,0.000000,NaN,68.30
3,1.0,1.0,10001366.0,0.0,0.0,114.80,BD059,0.0,184.0,199.0,...,0.661361,0.414647,0.217020,1.0,0.420019,-9.352077,0.000000,0.000000,42.358959,38.42
4,1.0,1.0,10001427.0,1.0,1.0,101.92,BD069,0.0,184.0,199.0,...,0.641838,0.479098,0.442595,1.0,11.430373,4.864430,11.430373,4.864430,55.920870,50.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,1.0,1.0,10003441.0,0.0,0.0,122.16,BD256,0.0,184.0,199.0,...,0.610428,0.381890,0.420613,1.0,-1.209766,4.010976,0.000000,0.000000,NaN,67.00
113,1.0,1.0,10003444.0,1.0,1.0,112.96,BD259,0.0,184.0,199.0,...,0.646852,0.446392,0.236772,1.0,7.094824,-10.214739,7.094824,-10.214739,47.253718,37.56
114,1.0,1.0,10003544.0,1.0,1.0,106.52,BD264,0.0,184.0,199.0,...,0.709331,0.390420,0.181592,1.0,10.285849,7.470933,10.285849,7.470933,49.275832,27.08
115,1.0,1.0,10003545.0,1.0,1.0,NaN,BD265,0.0,184.0,199.0,...,0.716776,0.464037,0.239919,1.0,6.818804,-5.721112,6.818804,-5.721112,48.927309,30.12


In [6]:
 
full_df['dxgroup'].value_counts()
full_df.to_csv('../data_processed/df_all_brainage_2025.csv', index=False)
